# Mission 3: Orchestrate SQL + AI workflows

This mission advances your skills by teaching you how to orchestrate complex workflows that combine SQL database operations with AI capabilities. You'll learn to build intelligent, multi-step processes that seamlessly integrate data retrieval, transformation, and AI-powered decision making.

## Overview
Learn how to design and implement sophisticated AI workflows that coordinate multiple database queries, embedding operations, and language model calls to solve complex business problems requiring multiple steps and decision points.


## Prerequisites
1. Mission 1 completed (embeddings and vector search)
1. Mission 2 completed (RAG implementation)
1. Embedding and language model access

In [1]:
#r "nuget: Microsoft.SqlServer.Server, 1.0.0"
#r "nuget: Microsoft.Data.SqlClient, 7.0.0-preview3.25342.7"
#r "nuget: DotNetEnv"
#r "nuget: Azure.Core, 1.50.0"
#r "nuget: Azure.AI.OpenAI, 2.8.0-beta.1"
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"



using System;
using System.Data;
using Microsoft.Data.SqlClient;


var queryString = "  SELECT TOP 10 * FROM dbo.walmart_ecommerce_product_details AS w";
DotNetEnv.Env.Load("../.env");  // Load from parent folder

var connectionString = Environment.GetEnvironmentVariable("SERVER_CONNECTION_STRING");
var endpoint = Environment.GetEnvironmentVariable("MODEL_ENDPOINT_URL");
var aoaiEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("MODEL_API_KEY");

using (SqlConnection connection = new SqlConnection(connectionString))
{
    SqlCommand command = new SqlCommand(queryString, connection);
    command.Connection.Open();
    SqlDataReader reader = command.ExecuteReader();
    while (reader.Read())
        {
            for (int i = 0; i < reader.FieldCount; i++)
            {
                Console.Write($"{reader[i]}\t");
            }
            Console.WriteLine();
        }
}

Installed Packages Azure.AI.OpenAI, 2.8.0-beta.1 Azure.Core, 1.50.0 DotNetEnv, 3.1.1 Microsoft.Data.SqlClient, 7.0.0-preview3.25342.7 Microsoft.SemanticKernel, 1.70.0 Microsoft.SemanticKernel.Agents.Core, 1.70.0 Microsoft.SqlServer.Server, 1.0.0

1	00068601cf9047122e9a9f8b300a763b	2019-03-22 17:56:41 +0000	https://www.walmart.com/ip/Ebe-Men-Black-Rectangle-Half-Rim-Spring-Hinge-Eyewear-Reading-Glasses-2036/199534909	Ebe Men Black Rectangle Half Rim Spring Hinge Eyewear Reading Glasses 2036	Count on EBE for all of your eye correction needs this light weight sleek looking pair of glasses are something to be won. We guarantee you will love the style of these frames with the sharply colored temples on the side of your face give a great fashion look and a great comfort your face we guarantee your satisfaction.| Always custom made to your individual order specifications in our FDA Listed Lab with the remarkable Optical grade Custom frames and Lenses available. Better style better grade and always less than retail. Rx lens included! Glasses come with a Hard protective case and micro fiber cleaning cloth. Very light and strong frame with Ebe signature comfort fit bridge for easy all day use without hot spots. We design and produce all 

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
                "gpt-4.1",
                aoaiEndpoint!,
                apiKey!
                );
var kernel = builder.Build();

ChatCompletionAgent agent =
    new()
    {
        Name = "SK-Agent",
        Instructions = "You are a helpful assistant.",
        Kernel = kernel,
    };

await foreach (AgentResponseItem<ChatMessageContent> response 
    in agent.InvokeAsync("Write a haiku about Semantic Kernel."))
{
    Console.WriteLine(response.Message);
}


Thoughts weave, skills combine  
Semantic patterns emerge—  
Kernel guides the mind.


In [3]:
using System.ComponentModel;
using Microsoft.SemanticKernel; 
 

    public string findProducts(string searchTerm)
    {
        var results = new List<string>();
        
        using (var connection = new SqlConnection(connectionString))
        {
            connection.Open();
            
            using (var command = new SqlCommand())
            {
                command.Connection = connection;
                command.CommandText = @"
                    SET NOCOUNT ON;
                    DECLARE @out nvarchar(max);
                    EXEC [dbo].[get_similar_items] @inputText = @searchTerm, @error = @out OUTPUT;
                    SELECT @out AS error;";
                
                command.Parameters.AddWithValue("@searchTerm", searchTerm);
                
                using (var reader = command.ExecuteReader())
                {
                    do
                    {
                        while (reader.Read())
                        {
                            var row = new List<string>();
                            for (int i = 0; i < reader.FieldCount; i++)
                            {
                                row.Add(reader[i]?.ToString() ?? "");
                            }
                            results.Add(string.Join(", ", row));
                        }
                    } while (reader.NextResult());
                }
            }
        }
        
        return string.Join("\n", results);
    }


// Register the plugin with the kernel
// Register using CreateFromMethod
kernel.Plugins.AddFromFunctions("ProductFunctions", new[]
{
    KernelFunctionFactory.CreateFromMethod(findProducts, "find_products", "Search the product catalog for items matching keywords.")
});


// Test the function
Console.WriteLine(findProducts("laptop"));

ChatCompletionAgent productAgent = new()
{
    Name = "Product-Agent",
    Instructions = """
        You are a product assistant who helps users find the best products available in the catalog.
        IMPORTANT: Work with whatever information the user provides and make reasonable assumptions when needed.
        Interpret the user's request directly, then use the find_products tool to get relevant products.
        Use only the information available in the provided JSON to answer the question.
        Don't return products that are not relevant with the ask or that don't comply with user request.
        If no products are provided, don't return any result.
        """,
    Kernel = kernel,
        Arguments = new KernelArguments(
        new PromptExecutionSettings 
        { 
            FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() 
        })
    
};

await foreach (var response in productAgent.InvokeAsync("I'm looking for a gaming laptop with high performance."))
{
    Console.WriteLine(response.Message);
}

[{"id":6532,"name":"Lenovo THINKPAD T440P 20AN0072IX Laptop Screen 14 SLIM LED BOTTOM RIGHT WXGA HD","description":"Lenovo THINKPAD T440P 20AN0072IX Laptop Screen 14 SLIM LED BOTTOM RIGHT WXGA HD|Please match your original screen's BACKLIGHT TYPE, SIZE, and RESOLUTION before you purchase this item, because some laptop models come with more than one type of screens."},{"id":10159,"name":"Refurbished Dell Latitude E6420 Laptop i5 Dual-Core 8GB 1TB Win 7 Pro B v.BB","description":"Dell Latitude E6420 Laptop 2.50GHz Minimum i5 Dual-Core Processor Speed 8GB 1TB Win 7 Pro Professionally Reconditioned with Warranty - Moderate Signs of Use Cosmetic Condition: Fair Condition - These laptops have moderate cosmetic scratches or scuffs on the bezel and casing and show signs of use on keyboard, trackpad and other highly utilized parts. The expansion\/SD slot covers may not be present. The screen may have no more than 2 of the following: light scratches, 1-2 dead pixels, or bruises no larger than a 